<a href="https://colab.research.google.com/github/SewoongPark/SeSac_study_repo/blob/main/%EB%8C%93%EA%B8%80_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

중고차 사이트 1


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

total_list = []
for page_num in range(1, 8):
  url = f'http://www.encar.com/mocha.do?method=listData&pageNo={page_num}&chapcd=SU&ctgrcd=&mnfccd=&mdlgroupcd=&mdlcd=&year='
  resp = requests.get(url)
  json_dic = resp.json()
  total_list.append(json_dic[0]['resultList'])

info = [total for i in range(0, 7) for total in total_list[i]]
info_df = pd.DataFrame(info)
info_df.drop('pic', axis = 1, inplace = True)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
info_df

,mnfcnm,cmmtwk,idbid,cmmtst,mdlnm,rat
0,벤츠,hoxy 오너? 쇼퍼?,3602,이보다 더 편안할 수 있을까?,S-클래스 W223,100
1,BMW,차이점이 뭐예요? N번째 설명 중,3343,3시리즈보다 윗급으로 보임,4시리즈 (F32),100
2,포르쉐,돈 값하는 차,3282,나(911) 자신과의 싸움,911 (992),100
3,도요타,카니발 오너가 부릅니다 '부럽지가 않어',3603,카니발 타는 아빠들의 부러움을 살 수 있다,알파드 4세대,90
4,현대,소형 SUV 잖아!,3462,소형 SUV 맞아?,코나 (SX2),90
...,...,...,...,...,...,...
205,쉐보레(GM대우),한심한 실내 질감,51,최고의 하체,스파크,40
206,현대,역대 최악의 고속안정성,48,2020년까지 버틸 만한 디자인,아반떼 MD,40
207,현대,심각한 차체 부식,102,운전 익힐 때 가장 좋음,뉴 아반떼 XD,30
208,르노코리아(삼성),이거 아직도 파는 거야?,88,르노 메간의 르노삼성 버전,뉴SM3,30


In [4]:
info_df.drop(['cmmtwk', 'cmmtst', 'rat'], axis = 1, inplace = True)

In [5]:
df_company = info_df['mnfcnm']
df_carId = info_df['idbid']
df_carName = info_df['mdlnm']

df_company.to_csv("company.csv")
df_carId.to_csv("cadId.csv")
df_carName.to_csv("carName.csv")

In [6]:
info_df

,mnfcnm,idbid,mdlnm
0,벤츠,3602,S-클래스 W223
1,BMW,3343,4시리즈 (F32)
2,포르쉐,3282,911 (992)
3,도요타,3603,알파드 4세대
4,현대,3462,코나 (SX2)
...,...,...,...
205,쉐보레(GM대우),51,스파크
206,현대,48,아반떼 MD
207,현대,102,뉴 아반떼 XD
208,르노코리아(삼성),88,뉴SM3


In [7]:
info_df.to_csv('encar_csv.csv', encoding = 'utf-8-sig')

댓글 가져오기

In [ ]:
commentList, idList, carIdList = [], [], []
for cid in df_carId:
  url2 = f'http://www.encar.com/mocha/mochacontents.do?method=searchModelDetailSummary&idbid={cid}'
  resp2 = requests.get(url2)
  soup = BeautifulSoup(resp2.content)
  comments = soup.select('div.box_comment p')
  ids = soup.select('div.box_comment span.txt_user')
  carIdList.append(int(cid)) #차량 id 담기

  for comment in comments:
    commentList.append(comment.text) #댓글 담기
  for id in ids:
    idList.append(id.text.split(" ")[0]) #user id담기


In [ ]:
df_comment = pd.DataFrame(commentList, columns = ['댓글'])
df_id = pd.DataFrame(idList)
df_carId = pd.DataFrame(carIdList)


df_comment['ID'] = df_id
df_comment['carId'] = df_carId

In [ ]:
df_comment.fillna("정보없음", inplace = True)

In [ ]:
df_comment["댓글"] = df_comment["댓글"].str.replace(pat=r'[^\w]', repl=' ', regex=True)

In [ ]:
df_comment['댓글'].to_csv('comment', 'comment.csv')

In [ ]:
df_comment.to_csv('엔카_댓글csv.csv', encoding='utf-8-sig')

중고차 사이트2 (autobell)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://autobell.co.kr/api/buycar/selectGeneralList.do'
payload = {"isMobile":"N","cho":["1","2"],"carType":[],"crMnfcCd":"","crMnfcNm":"","crMdlCd":"","crMdlNm":"","crDtlMdlCd":[],"crDtlMdlNm":[],"crClsCd":[],"crClsNm":[],"crDtlClsCd":[],"crDtlClsNm":[],"selectedModels":[],"yearRange":{"min":1990,"max":2024,"labelmax":"최대","labelmin":"최소"},"distanceRange":{"min":0,"max":990000,"labelmax":"최대","labelmin":"최소"},"priceRange":{"min":0,"max":9999999,"labelmax":"최대","labelmin":"최소"},"loc":[],"carOption":[],"color":[],"fuel":[],"mss":[],"svc":[],"brndGrpTp":"","brndGrpId":"","brndCmpnId":"","mrktCmplxSeqNo":"","acdt":[],"searchTerm":"","currentPage":1,"countPerPage":3125,"countPerMore":3125,"type":"normal","viewType":"0","tab":"0","subTab":"","detailTab":"","listType":"card","homeSvc":"N","order":"upd_dt","selected":[],"selectedList":[],"advConditionStack":[],"smpConditionStack":[],"filterTab":"0"}
resp = requests.post(url, json=payload) #POST 요청으로 데이터 가져오기
#어려웠던 점: 해당 url로 데이터 크롤링을 시도했는데 405 error 발생, payload 값을 받아와 POST요청으로 해결함.
json_dict = resp.json()

In [ ]:
#개별 차량 페이지에서 데이터를 가져오기 위해 productid값을 뽑아 list에 담기
prdIdList = []
for i in json_dict['data']['prdLst'][5:]:
  prdIdList.append(i['dlrPrdId'])


In [ ]:
len(prdIdList)

3120

In [ ]:
likeList, viewList = [],[] #좋아요, 조회수 정보 담기
for prdId in prdIdList:
  html = f'https://autobell.co.kr/buycar/buyCarDetailType?dlrPrdId={prdId}'
  resp2 = requests.get(html)
  soup = BeautifulSoup(resp2.content)
  likes = soup.select("label")[1::2] #label tag안에 여러 정보가 담겨있어서, 필요한 정보만 슬라이싱해서 선택
  views = soup.select("label")[2::2]

  for like in likes:
    likeList.append((like.text))

  for view in views:
    viewList.append((view.text))


In [ ]:
df_like = pd.DataFrame(likeList)
df_like['view'] = viewList
df_like.columns = ['좋아요', '조회수']
df_like

,좋아요,조회수
0,5,"1,041"
1,4,429
2,6,862
3,0,19
4,0,66
...,...,...
3115,0,4
3116,1,4
3117,0,4
3118,0,4


In [ ]:
ids = []
conv = ''
for i in json_dict['data']['prdLst'][5:]:
 ids.append(i['dlrPrdId'])
 #어려웠던 점: json_dict의 시작점에 관한 문제, 원하는 key값을 못찾는 keyerror발생, 시작점을 5번째부터 for문을 돌리는 것으로 문제 해결.
li_engine, li_mission, li_year, li_cc, li_accident = [], [], [], [], []
for id in ids:
  url = f'https://autobell.co.kr/buycar/buyCarDetailType?dlrPrdId={id}'
  resp = requests.post(url)
  soup = BeautifulSoup(resp.content)

  #엔진, 변속기, 가격, 연식, 배기량 등의 정보를 td tag에서 index로 선택하여 가져오기(이 모든 정보들이 td태그 하나에 들어가 있기 때문)
  engines = soup.select("td")[19]
  transmissions = soup.select("td")[20]
  years = soup.select("td")[22]
  ccs = soup.select("td")[23]
  accidents = soup.select("td")[24]

  for newinfo in engines:
    li_engine.append(engines.text)
    li_mission.append(transmissions.text)
    li_year.append(years.text)
    li_cc.append(ccs.text)
    li_accident.append(accidents.text)


In [ ]:
carNameList, companyList, priceList = [], [], [] #차 이름, 제조사, 가격 정보 담기
for i in json_dict['data']['prdLst'][5:]:
  i['slAmt'] = str(i['slAmt'])
  if len(i['slAmt']) >= 4: #4자릿수 이상 가격에 ',' 붙이기
    i['slAmt'] = i['slAmt'][0] + "," + i['slAmt'][1:]
  carNameList.append(i['crNm'])
  companyList.append(i['mnfcNm'])
  priceList.append(i['slAmt'])

In [ ]:
import pandas as pd

df = pd.DataFrame(companyList)

df['carName'] = carNameList
df['price'] = priceList
df['engine'] = li_engine
df['mission'] = li_mission
df['cc'] = li_cc
df['year'] = li_year
df['accident'] = li_accident

df.columns = ['company', 'carName','price','engine','mission','cc','year','accident']

In [ ]:
df['like'] = df_like['좋아요']
df['view'] = df_like['조회수']
df

,company,carName,price,engine,mission,cc,year,accident,like,view
0,폭스바겐,폭스바겐 티구안 R라인 2.0 TDI 블루모션,"1,650",디젤,자동(오토),1968 cc,2015,무사고,5,"1,041"
1,현대,현대 코나 프리미엄 가솔린 1.6 터보,"1,690",가솔린,자동(오토),1591 cc,2018,무사고,4,429
2,현대,현대 LF쏘나타 뉴 라이즈 가솔린 2.0 프리미엄 스페셜,"1,600",가솔린,자동(오토),1999 cc,2019,무사고,6,862
3,쌍용,KG모빌리티(쌍용) 올 뉴코란도 딜라이트 플러스 e-XDi160 LET,"1,360",디젤,자동(오토),1597 cc,2019,무사고,0,19
4,현대,현대 쏘나타(DN8) 프리미엄 패밀리 스마트스트림 가솔린 2.0,"2,170",가솔린,자동(오토),1999 cc,2020,무사고,0,66
...,...,...,...,...,...,...,...,...,...,...
3115,기아,기아 K8 2.5 GDI 노블레스,"3,560",있음,있음,,없음,없음,0,4
3116,현대,현대 쏘나타(DN8) 스마트스트림 LPi 2.0 렌터카 스타일,"1,770",있음,있음,,없음,없음,1,4
3117,제네시스,제네시스 더 올뉴G80 가솔린 3.5 터보 AWD 스포츠팩,"6,930",있음,있음,,없음,없음,0,4
3118,현대,현대 팰리세이드 가솔린 3.8 AWD VIP,"4,560",있음,있음,,없음,없음,0,4


In [ ]:
df['accident'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3120 entries, 0 to 3119
Series name: accident
Non-Null Count  Dtype 
--------------  ----- 
3120 non-null   object
dtypes: object(1)
memory usage: 24.5+ KB


In [ ]:
df.to_csv('autobell.csv', encoding='utf-8-sig')